In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers

inputs = tf.keras.Input(shape=(32, 32, 1), name="input_image")

# Shared
x = layers.Conv2D(32, 3, padding="same", activation="relu")(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)

x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)

shared = layers.Activation("linear", name="shared_block")(x)

# Task A (10-class)
a = layers.Conv2D(128, 3, padding="same", activation="relu")(shared)
a = layers.BatchNormalization()(a)
a = layers.GlobalAveragePooling2D()(a)
a = layers.Dense(
    128, activation="relu",
    kernel_regularizer=regularizers.l2(1e-4)
)(a)
a = layers.Dropout(0.5)(a)
output_A = layers.Dense(10, activation="softmax", name="output_A")(a)

# Task B (32-class)
b = layers.Conv2D(128, 3, padding="same", activation="relu")(shared)
b = layers.BatchNormalization()(b)
b = layers.Conv2D(128, 3, padding="same", activation="relu")(b)
b = layers.BatchNormalization()(b)
b = layers.GlobalAveragePooling2D()(b)
b = layers.Dense(
    256, activation="relu",
    kernel_regularizer=regularizers.l2(1e-4)
)(b)
b = layers.Dropout(0.5)(b)
output_B = layers.Dense(32, activation="softmax", name="output_B")(b)

# Task C (Regression)
c = layers.Conv2D(64, 3, padding="same", activation="relu")(shared)
c = layers.BatchNormalization()(c)
c = layers.GlobalAveragePooling2D()(c)
c = layers.Dense(
    64, activation="relu",
    kernel_regularizer=regularizers.l2(1e-4)
)(c)
output_C = layers.Dense(1, activation="linear", name="output_C")(c)

model = tf.keras.Model(
    inputs=inputs,
    outputs=[output_A, output_B, output_C]
)

# Load trained weights saved from Option B
model.load_weights("submission_s4021255_s3978616.h5")



In [ ]:
# Load dataset in the original format (X shape: N x 32 x 32)
data = np.load("dataset_dev_3000.npz")
X = data["X"]

def predict_multitask_preprocessing(model, X_input):
    X_input = np.array(X_input, dtype=np.float32)

    # Ensure batch dimension
    if X_input.ndim == 2:
        X_input = np.expand_dims(X_input, axis=0)
    elif X_input.ndim != 3:
        raise ValueError("Input must be shape (32,32) or (N,32,32)")

    # Preprocessing: log1p + max normalization + channel expansion
    X_log = np.log1p(X_input)
    max_log_value = X_log.max()
    X_norm = X_log / max_log_value
    X_norm = np.expand_dims(X_norm, axis=-1)  # (N, 32, 32, 1)

    pred_A, pred_B, pred_C = model.predict(X_norm)
    return pred_A, pred_B, pred_C

# Prediction call
pred_A, pred_B, pred_C = predict_multitask_preprocessing(model, X)

print("pred_A shape:", pred_A.shape)
print("pred_B shape:", pred_B.shape)
print("pred_C shape:", pred_C.shape)
print("pred_A first 5:", np.argmax(pred_A, axis=1)[:5])
print("pred_B first 5:", np.argmax(pred_B, axis=1)[:5])
print("pred_C first 5:", pred_C.reshape(-1)[:5])

